# Monitoring
The Monitor class allows you to track defined parameters, visualize their time series, and implement custom reactions if they go outside of defined thresholds. Let's say we have some function which measures and returns a voltage, which for the purposes of this tutorial we will generate from a normal distribution:

In [ ]:
import numpy as np
def read_voltage():
    return np.random.normal(1, 0.05)

It's very simple to define a Monitor to watch this voltage:

In [ ]:
from watchdog import Monitor
m = Monitor(visualize=True)
m.watch(read_voltage)
m.start(period=1)
m.plot()

Note that the Monitor.start_periodic method uses Python's sched module for event scheduling to avoid accumulated drifts. For example, if your check() method takes 50 ms to return and you call it in a loop with a 1s delay, the true time between checks will actually be 1.05 s. Using the scheduler allows drift-free checking; however, you should make sure that the check() call always returns in a time shorter than your chosen period!

## Watching new variables
New variables can be added dynamically while the monitor is running:

In [ ]:
def read_current():
    return np.random.normal(0.1, 0.01)
m.watch(read_current, name='current')

## Reactions

When you specify a variable to watch, you can pass a threshold tuple defining the "good" range for the variable. You can also pass a function to be called when the variable goes outside of the threshold:

In [ ]:
def read_noisy_voltage():
    return np.random.normal(0.5, 0.1)

def alert():
    print('Noisy voltage below threshold!')
    
m.watch(read_noisy_voltage, threshold=(0.4, None), reaction = alert)

To stop the monitoring, call the Monitor.stop() method:

In [ ]:
m.stop()

## Triggering
Monitoring can be synced to some other process through software triggers, e.g. TTL pulses acquired with a DAQ board. Just define a method which returns as soon as a trigger is received and pass the trigger into the Monitor.start() method. Let's simulate a trigger arriving once per second:

In [ ]:
import time
def trigger():
    time.sleep(1)
    return

m.start(trigger=trigger)

In [ ]:
m.stop()

## Subscribing to monitor feeds
By default, the Monitor class publishes new data using the ZeroMQ protocol on 127.0.0.1:1105 (these default settings can be changed through the ``address`` and ``port`` keyword arguments). Other classes can subscribe to this feed using the Subscriber class:

In [ ]:
from watchdog import Subscriber
        
s = Subscriber()
s.get()